# Tutorial 1 - RNN Sequence classifier

In this notebook, we will predict the winner of a basketball game based on the scores observed in the first 3 quarters of the game. Each column represents the beginning of a minute during the game. (There are 12 minutes in each quarter. There are  3 quarters in the data, so we have 36 columns as input variables in chronological order.) The values captured in each column represent the score difference observed at that minute (home score minus away score)<br><br>

The last column `W` represents whether the home team (1) or the away team (0) won the game. This is the target variable. <br><br>
**Our unit of analysis is a single game.**

In [ ]:
# Common imports
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

## Get the data

In [ ]:
data = pd.read_csv("basketball.csv")

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
y = data['W']
x = data.drop('W', axis=1)

## Split the data

In [ ]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.3)

## Data Transformation

In [ ]:
#Target variables need to be an array with integer type
train_y = np.array(train_y)
test_y = np.array(test_y)

train_y = train_y.astype(np.int32)
test_y = test_y.astype(np.int32)



In [ ]:
#Check the first 10 values of the train_y data set
train_y[0:10]

In [ ]:
#Convert input variables to a 2-D array with float data type
train_x= np.array(train_x)
test_x= np.array(test_x)

train_x = train_x.astype(np.float32)
test_x = test_x.astype(np.float32)

In [ ]:
train_x

In [ ]:
#Keras expects a different input format:
#Data needs to have 3 dimensions

train_x = np.reshape(train_x, (train_x.shape[0], train_x.shape[1], 1))
test_x = np.reshape(test_x, (test_x.shape[0], test_x.shape[1], 1))



In [ ]:
train_x.shape, train_y.shape

In [ ]:
train_x

# Baseline Accuracy

In [ ]:
data['W'].value_counts()/len(data)

# A normal (cross-sectional) NN

This model assumes that the data is NOT a time-series data set. It treats the data as cross-sectional and the columns being independent of each other.

In [ ]:
model = keras.models.Sequential([
    
    keras.layers.Flatten(input_shape=[36, 1]),
    keras.layers.Dense(20, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
    
])

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = tf.keras.optimizers.Nadam(lr=0.01)

# If multiclass, use "sparse_categorical_crossentropy" as the loss function
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])


history = model.fit(train_x, train_y, epochs=50,
                    validation_data=(test_x, test_y))

In [ ]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

In [ ]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


# Simple RNN

## One layer

In [ ]:
n_steps = 36
n_inputs = 1


model = keras.models.Sequential([
    
    keras.layers.SimpleRNN(1, activation='sigmoid' , input_shape=[n_steps, n_inputs]),
    
])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping


earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

callback = [earlystop]

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = tf.keras.optimizers.Nadam(lr=0.01)

# If multiclass, use "sparse_categorical_crossentropy" as the loss function
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])


history = model.fit(train_x, train_y, epochs=50,
                    validation_data=(test_x, test_y), callbacks=callback)

In [ ]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

In [ ]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


In [ ]:
# Predictions are probabilities.

predictions = model.predict(test_x)

In [ ]:
# Rounding the probabilities determines 1 or 0

np.round(predictions)

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(test_y, np.round(predictions))

## Add an output layer as Dense

This is nearly same as keeping the last layer as `SimpleRNN`. Though, it makes faster computations. Also, SimpleRNN uses only the tanh activation function. By changing it to Dense, you can select another activation function.



In [ ]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    
    keras.layers.SimpleRNN(2, input_shape=[n_steps, n_inputs]),

    #If multi-class, use this:
    #tf.keras.layers.Dense(n_classes, activation="softmax")
    
    keras.layers.Dense(1, activation='sigmoid')
    
])

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = tf.keras.optimizers.Nadam(lr=0.01)

# If multiclass, use "sparse_categorical_crossentropy" as the loss function
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])


history = model.fit(train_x, train_y, epochs=20,
                    validation_data=(test_x, test_y), callbacks=callback)

In [ ]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

In [ ]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


# Deep RNN

**Be careful: when stacking RNN layers, you have to set "return_sequences" to True. This enables the layer to send a "sequence" of values to the next layer -- jut like how it uses a sequence of values for training.**

**Since the last layer is DENSE, it can't take sequence data. Therefore, you CANNOT return sequences from the previous layer. So, remove `return_sequences` from previous layer.**

In [ ]:
n_steps = 36
n_inputs = 1


model = keras.models.Sequential([
    keras.layers.SimpleRNN(3, return_sequences=True, input_shape=[n_steps, n_inputs] ),
    keras.layers.SimpleRNN(3, return_sequences=True),
    keras.layers.SimpleRNN(3), 
    keras.layers.Dense(1, activation='sigmoid')
])


In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(lr=0.001)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=20,
                   validation_data = (test_x, test_y), callbacks=callback)

In [ ]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

# LSTM - One layer

In [ ]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    
    keras.layers.LSTM(1, activation='sigmoid' , input_shape=[n_steps, n_inputs])
])

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(lr=0.001)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=20,
                   validation_data = (test_x, test_y), callbacks=callback)

In [ ]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

In [ ]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


In [ ]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


# LSTM - with Dense layer

In [ ]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    
    keras.layers.LSTM(1, input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(lr=0.001)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=20,
                   validation_data = (test_x, test_y), callbacks=callback)

In [ ]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

In [ ]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


## Deep LSTM

In [ ]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.LSTM(2, return_sequences=True, input_shape=[n_steps, n_inputs]),
    keras.layers.LSTM(2, return_sequences=True),
    keras.layers.LSTM(1),
    keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(lr=0.001)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=20,
                   validation_data = (test_x, test_y), callbacks=callback)

In [ ]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

In [ ]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


# GRU Cells

In [ ]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.GRU(2, return_sequences=True, input_shape=[n_steps, n_inputs]),
    keras.layers.GRU(2, return_sequences=True),
    keras.layers.GRU(2, return_sequences=True),
    keras.layers.GRU(1, activation='sigmoid')
])

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(lr=0.001)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=20,
                   validation_data = (test_x, test_y), callbacks=callback)

In [ ]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

In [ ]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


# Conv1D

In [ ]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.Conv1D(filters=20, kernel_size=3, strides=2, padding="valid", input_shape=[n_steps, n_inputs]),
    keras.layers.Conv1D(filters=20, kernel_size=3, strides=1, padding="valid", dilation_rate=2),
    keras.layers.LSTM(20, return_sequences=True),
    keras.layers.LSTM(20),
    keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(lr=0.001)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=20,
                   validation_data = (test_x, test_y), callbacks=callback)

In [ ]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

In [ ]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
